In [64]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os, sys
import re

#prediction and ground truth paths
pr_path = "/Users/rouven/Documents/SCPD/CS230/CS230_Project/Temp16_mini_b16_d2/"
#pr_path = "/Users/rouven/Documents/SCPD/CS230/CS230_Project/testing3/prediction/"
gt_path = "/Users/rouven/Documents/SCPD/CS230/CS230_Project/MAR18/test_labels/"
#gt_path = "/Users/rouven/Documents/SCPD/CS230/CS230_Project/testing3/ground_truth/"
out_path = "/Users/rouven/Documents/SCPD/CS230/CS230_Project/Temp16_mini_b16_d2/out/"
#out_path = "/Users/rouven/Documents/SCPD/CS230/CS230_Project/testing/"

pr_dirs = os.listdir( pr_path )
gt_dirs = os.listdir( gt_path )

def compute_IoU_mPA(pr, gt):
    
    EPS = 1e-12
    n_classes = 4

    #Classwise Intersection over Union IoU
    class_wise_IoU = np.zeros(n_classes)
    for cl in range(n_classes):
        intersection = np.sum((gt == cl)*(pr == cl))
        union = np.sum(np.maximum((gt == cl), (pr == cl)))
        iou = float(intersection)/(union + EPS)
        class_wise_IoU[cl] = iou
            
    #Classwise Pixel Accuracy mPA
    class_wise_mPA = np.zeros(n_classes)
    for cl in range(n_classes):
        intersection = np.sum((gt == cl)*(pr == cl))
        total = np.sum((gt == cl))
        mpa = float(intersection)/(total + EPS)
        class_wise_mPA[cl] = mpa
    
    return class_wise_IoU, class_wise_mPA
    

def compute_IoU():
    
    IoU = np.empty(4)
    mPA = np.empty(4)
    
    for item in pr_dirs:
        if item == '.DS_Store' or item == 'distribution.txt':
            continue
        if os.path.isfile(pr_path+item):
            f, e = os.path.splitext(out_path+item)
            pr = Image.open(pr_path+item)
            gt = Image.open(gt_path+item)
            pr = pr.convert('RGBA')
            gt = gt.convert('RGBA')

            #Get RGB values from predictions
            pr_data = np.array(pr)   # "data" is a height x width x 3 numpy array
            red, green, blue, alpha = pr_data.T # Temporarily unpack the bands for readability
            background = (red == 20) & (green == 215) & (blue == 197) 
            facet_100 = (red == 207) & (green == 248) & (blue == 132)
            facet_111 = (red == 183) & (green == 244) & (blue == 155)
            facet_110 = (red == 144) & (green == 71) & (blue == 111)
            rest = (np.logical_not(background) & np.logical_not(facet_100) & np.logical_not(facet_111) & np.logical_not(facet_110))
            #print(np.sum(rest))
            
            #Convert to decimal
            pr_data[..., :-1][rest.T] = 0
            pr_data[..., :-1][background.T] = 0 
            pr_data[..., :-1][facet_100.T] = 1 
            pr_data[..., :-1][facet_111.T] = 2 
            pr_data[..., :-1][facet_110.T] = 3 
            
            #Get RGB values from ground truth
            gt_data = np.array(gt)   # "data" is a height x width x 3 numpy array
            red, green, blue, alpha = gt_data.T # Temporarily unpack the bands for readability
            facet_100 = (red == 255) & (green == 0) & (blue == 1) 
            facet_111 = (red == 0) & (green == 255) & (blue == 2)
            facet_110 = (red == 255) & (green == 255) & (blue == 3)
            background = (red == 0) & (green == 0) & (blue == 0)
            #rest = (np.logical_not(background) & np.logical_not(facet_100) & np.logical_not(facet_111) & np.logical_not(facet_110))
            #print(np.sum(rest))
            print(np.sum(facet_111))
            
            #Convert to decimal
            gt_data[..., :-1][background.T] = 0 
            gt_data[..., :-1][facet_100.T] = 1 
            gt_data[..., :-1][facet_111.T] = 2 
            gt_data[..., :-1][facet_110.T] = 3       
            
            #Classwise IoU computation
            pr = pr_data
            gt = gt_data
 
            class_wise_IoU, class_wise_mPA = compute_IoU_mPA(pr, gt)
            #print(item, class_wise_IoU, class_wise_mPA)
    
            IoU = np.vstack((IoU, class_wise_IoU))
            mPA = np.vstack((mPA, class_wise_mPA))
               
            IoU.astype('float')
            IoU[IoU == 0] = np.nan
            #IoU = np.delete(IoU, (0), axis=0)
            #print(np.nanmean(IoU, axis=0))
    
            mPA.astype('float')
            mPA[mPA == 0] = np.nan
            #mPA = np.delete(mPA, (0), axis=0)
            #print(np.nanmean(mPA, axis=0))
            #print(mPA)
            
            #Image conversion and annotation
            out = Image.open(pr_path+item)
            out = out.convert('RGBA')
            out_data = np.array(out)   # "data" is a height x width x 4 numpy array
            red, green, blue, alpha = out_data.T # Temporarily unpack the bands for readability
            background = (red == 20) & (green == 215) & (blue == 197) 
            facet_100 = (red == 207) & (green == 248) & (blue == 132)
            facet_111 = (red == 183) & (green == 244) & (blue == 155)
            facet_110 = (red == 144) & (green == 71) & (blue == 111)
            
            #Convert predictions to RGBA
            out_data[..., :-1][background.T] = (128, 128, 128)
            out_data[..., :-1][facet_100.T] = (255, 0, 86)
            out_data[..., :-1][facet_111.T] = (0, 255, 2)
            out_data[..., :-1][facet_110.T] = (0, 102, 255)
            
            im2 = Image.fromarray(out_data)
            im2 = im2.convert('RGB') #Rouven
            perf = np.round(class_wise_IoU*100, decimals=0)
            ImageDraw.Draw(
                im2  # Image
                ).text(
                    (0, 0),  # Coordinates
                #str(perf),  # Text
                (0, 0, 0)  # Color
                )
            im2.save(f + 'new.png')
    print(np.nanmean(IoU, axis=0))
    print(np.nanmean(mPA, axis=0))

    #with open(path + "distribution.txt", "w") as text_file:
        #text_file.write("black: %f red: %f green: %f yellow: %f other: %f pixels: %d" % distribution)
compute_IoU()

0
7787
198
260
7166
242
0
9034
194
187
0
185
192
0
148
0
3476
0
7617
3072
0
232
235
240
8708
290
7540
431
841
7159
5344
7576
0
164
7446
202
8398
249
7474
3525
254
8766
0
0
7471
7571
0
8620
262
279
0
0
0
0
160
574
5884
0
4882
0
0
8535
190
0
6297
266
0
6622
0
263
235
0
0
8002
0
0
5592
243
8630
1790
7794
8916
6758
8622
8084
177
210
8571
6202
7618
192
231
0
8125
0
0
199
7822
6956
156
0
269
8611
200
193
72
9004
261
7779
6490
0
242
267
0
214
142
7099
163
4911
210
4220
216
179
174
7949
0
0
5287
390
207
6444
101
6249
0
0
7876
7427
194
0
227
7210
6320
7239
7592
6575
0
6309
0
0
244
0
218
0
219
201
7444
254
248
0
227
198
7196
189
235
0
8008
0
342
0
0
8605
0
172
0
210
7113
195
3211
0
0
156
0
0
7962
237
0
234
8740
0
0
8884
8201
181
224
6760
212
229
0
7964
0
245
0
164
0
0
8909
243
0
0
252
7737
249
0
6710
0
0
7718
0
221
0
199
199
0
8749
5012
8928
0
0
231
193
202
572
216
7730
4715
8516
0
0
217
1842
0
177
0
224
259
7899
0
1394
0
7945
[0.87139845 0.48900448 0.94148365 0.54409977]
[0.79186663 0.54660464 

In [54]:
0.33*3/2

0.495

In [58]:
(0.62894441+0.48900448+0.44114344+0.54409977)/4

0.5257980250000001

In [63]:
250/(96*96)

0.027126736111111112